## Подготовка

In [1]:
pip install spaCy

In [2]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas jupyter pandarallel 

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install wordcloud

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [6]:
import re
import nltk
import numpy as np
import pandas as pd
import spacy
import matplotlib.pyplot as plt

from datetime import datetime
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
from tqdm import tqdm
tqdm.pandas()
from pandarallel import pandarallel   
tqdm.pandas(desc="progress")
pandarallel.initialize(progress_bar = True)
from wordcloud import WordCloud

nltk.download('stopwords')

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danii\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Загрузка данных

In [9]:
df_orig = pd.read_csv(r"C:\Users\danii\projects\project_vikishop\toxic_comments.csv")
df = df_orig.copy()
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [10]:
df = df.drop('Unnamed: 0', axis = 1)

### Поиск пропусков и дубликатов

In [11]:
print(f'Количество пропусков: \n{df.isna().sum()}\n\n'+
     f'Количество дубликатов: {df.duplicated().sum()}')

Количество пропусков: 
text     0
toxic    0
dtype: int64

Количество дубликатов: 0


Пропуски и дубликаты не обнаружены.

### Лемматизация исходных данных

In [12]:
def lemm(text): 
    nlp = spacy.load('en_core_web_sm')
    lemm_list = []
    
    for i in range(len(text)):
        doc = nlp(text[i])
        lemm_text = ' '.join([token.lemma_ for token in doc])
        lemm_list.append(lemm_text.lower())
        
    return lemm_list
    

In [13]:
print(datetime.now())
start = datetime.now()
df['lemm_text'] = lemm(df['text'])
print(f'--- {datetime.now() - start} ---')

2023-09-08 21:29:04.212170
--- 1:53:20.232670 ---


### Очистка лемматизированных текстов от лишних символов

In [14]:
def clear_text(text):
    clear_list = ' '.join(re.sub(r'[^A-Za-z]', ' ', text).split())
    return clear_list

In [15]:
df['cl_l_text'] = df['lemm_text'].progress_apply(clear_text)

progress: 100%|█████████████████████████████████████████████████████████████| 159292/159292 [00:14<00:00, 11004.85it/s]


## Обучение

### Разделение данных на обучающую и тестовую выборки

In [16]:
corpus = df['cl_l_text']
target = df['toxic']
train, test, target_train, target_test = train_test_split(corpus, target, test_size = 0.25,
                                                          random_state = 123, stratify = target) 
corpus_train = train.values
corpus_test = test.values

### Определение величины TF-IDF

In [17]:
stop_words = nltk.corpus.stopwords.words('english')

Tfid = TfidfVectorizer(stop_words = stop_words)
Tfid.fit(corpus_train)
tf_idf_train = Tfid.transform(corpus_train)
tf_idf_test = Tfid.transform(corpus_test)
names = Tfid.vocabulary_.keys()

### Обучение модели логистической регрессии

In [18]:
params = {'C': np.arange(1, 20, 2)}
model = LogisticRegression(random_state = 123)
grid = GridSearchCV(model, cv = 5, n_jobs = -1, param_grid = params, scoring = 'f1')
grid.fit(tf_idf_train, target_train)
print(grid.best_params_)
print('Значение метрики F1:', grid.cv_results_['mean_test_score'].mean())

{'C': 17}
Значение метрики F1: 0.76073225457165


C:\Users\danii\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Обучение модели LightGBM

In [19]:
print(datetime.now())
start = datetime.now()
model = LGBMClassifier(boosting_type = 'gbdt', random_state = 123)
param = {
    'n_estimators': range(100, 310, 100),   
    'learning_rate' : np.arange(0.1, 1, 0.3)
}

grid = GridSearchCV(model, param, cv = 5, scoring = 'f1', n_jobs = -1)
grid.fit(tf_idf_train, target_train)

print('Лучшие гиперпараметры:', grid.best_params_)
print('Метрика F1:', grid.cv_results_['mean_test_score'].mean())
print(f'--- {datetime.now() - start} ---')

2023-09-08 23:28:43.666268
Лучшие гиперпараметры: {'learning_rate': 0.4, 'n_estimators': 300}
Метрика F1: 0.7546645685205333
--- 0:38:21.454207 ---


### Обучение модели PassiveAggressiveClassifier

In [20]:
params = {'C': np.arange(0.01, 0.11, 0.01)}
model = PassiveAggressiveClassifier(random_state = 123)
grid = GridSearchCV(model, cv = 5, n_jobs = -1, param_grid = params, scoring = 'f1')
grid.fit(tf_idf_train, target_train)
print(grid.best_params_)
print('Значение метрики F1:', grid.cv_results_['mean_test_score'].mean())

{'C': 0.05}
Значение метрики F1: 0.7750515427892781


### Выбор лучшей модели и обучение на тестовой выборке 

Лучший результат метрики F1 на обучающей выборке был у модели PassiveAggressiveClassifier

In [21]:
model = PassiveAggressiveClassifier(random_state = 123, C = 0.05, n_jobs = -1)
model.fit(tf_idf_train, target_train)
predicted_train = model.predict(tf_idf_train)
predicted_test = model.predict(tf_idf_test)
print('Метрика F1 для обучающей выборки: ', f1_score(predicted_train, target_train))
print('Метрика F1 для тестовой выборки: ', f1_score(predicted_test, target_test))

Метрика F1 для обучающей выборки:  0.8820182596039342
Метрика F1 для тестовой выборки:  0.7897742725047593


## Выводы

В результате подготовки данных была выполнена загрузка исходных данных в датафрейм, была выполнена предобработка данных (поиск пропусков и дубликатов); была выполнена лемматизация текстовых данных; текста были очищены от лишних символов.

В ходе обучения данные были разделены на обучающую и тестовую выборки; было выполнено преобразование выборок в матрицы значений TF-IDF; эти матрицы выступли в роли признаков, по которым были обучены модели логистической регрессии, LightGBM, PassiveAggressiveClassifier. 

Для данных моделей было определено усредненное значение метрики F1 для обучающей выборки с применением кросс-валидации, таким образом была определена лучшая модель с оптимальными гиперпараметрами - PassiveAggressiveClassifier. Для данной модели было выполнено обучение и предсказание на тестовых данных и был определен финальный результат метрики F1 - 0.79.